In [8]:
pip install cvzone


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import cv2
import numpy as np
import cvzone
import random
import pandas as pd

pilotos = pd.read_csv("/Users/diegoc85/Challenge_python/formula_e_pilotos_2023_2024.csv", header = None)
pilotos

usuarios = []

# Cadastrar um novo usuário no app para acesso do Vision em tempo real
def cadastrar_usuarios(usuarios):
    print("---- CADASTRO ----")
    nome = input("Digite seu nome:")
    email = input("Digite seu e-mail:")
    senha = getpass.getpass("Digite sua senha:")
    usuarios.append({'Nome': nome, 'E-mail': email, 'Senha': senha})
    print('Cadastro realizado com sucesso!\n')
    print("-----------------------------------------")

    for indice, dicionario in enumerate(usuarios):
        print(f"\nCadastro {indice + 1}:")
        for chave, valor in dicionario.items():
            print(f"    {chave} : {valor}")
    print("\n")

# Logar na área exclusiva do Vision FE
def login(usuarios):
    tentativas = 3

    while tentativas > 0:
        print("----- LOGIN ----")
        email = input("E-mail:")
        senha = getpass.getpass('Senha:')
        for usuario in usuarios:
            if usuario['E-mail'] == email and usuario['Senha'] == senha:
                print(f"Login efetuado com sucesso, {usuario['Nome']}!")
                return True
        tentativas -= 1
        print(f"E-mail ou Senha incorretos. Tente novamente! Você tem mais {tentativas} tentativas.")
    print("Tentativas excedidas! Fim do programa.")
    return False

def menu_exclusivo(pilotos):
    print("\n---- Bem-vindo à Área Exclusiva do Vision FE! ----")

    while True:
        print("\n1) Acesse o Vision FE")
        print("2) Mostrar velocidade atual do carro rastreado")
        print("3) Posição atual do carro rastreado")
        print("4) Nome do piloto e da equipe do carro rastreado")
        print("5) Sair")

        opcao = input("\nEscolha uma das opções do Menu.\n")

        match opcao:
            case "1":
                print("\n\nAcessando o Vision FE\n\n")

                configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'  # config do modelo
                modelPath = 'frozen_inference_graph.pb'
                classesPath = 'coco.names'

                net = cv2.dnn_DetectionModel(modelPath, configPath)

                net.setInputSize(320, 320)
                net.setInputScale(1.0 / 127.5)
                net.setInputMean((127.5, 127.5, 127.5))
                net.setInputSwapRB(True)

                with open(classesPath, 'r') as f:
                    classesList = f.read().splitlines()

                # Usa a webcam como fonte de vídeo (índice 0 para a webcam padrão)
                video = cv2.VideoCapture(0)

                while True:
                    check, img = video.read()

                    if not check:
                        print("Erro ao capturar imagem da webcam")
                        break

                    img = cv2.resize(img, (1270, 720))

                    labels, confs, bboxs = net.detect(img, confThreshold=0.5)

                    bboxs = list(bboxs)
                    confs = list(np.array(confs).reshape(1, -1)[0])
                    confs = list(map(float, confs))

                    bboxsIdx = cv2.dnn.NMSBoxes(bboxs, confs, score_threshold=0.5, nms_threshold=0.3)

                    if len(bboxsIdx) != 0:
                        for x in range(0, len(bboxsIdx)):
                            bbox = bboxs[np.squeeze(bboxsIdx[x])]
                            conf = confs[np.squeeze(bboxsIdx[x])]
                            labelsID = np.squeeze(labels[np.squeeze(bboxsIdx[x])]) - 1
                            label = classesList[labelsID]

                            print(bbox, labelsID, conf)
                            x, y, w, h = bbox

                            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 3)
                            cvzone.putTextRect(img, f'{label} {round(conf, 2)}', (x, y - 10), colorR=(255, 0, 0), scale=1, thickness=2)

                    cv2.imshow('Imagem', img)

                    # Finalizar o programa pressionando a tecla 'q'
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                # Libera a captura da webcam e fecha as janelas
                video.release()
                cv2.destroyAllWindows()

            case "2":
                velocidade = random.randint(0, 320)
                print(f"\n\nA velocidade atual do carro rastreado é de {velocidade} km/h.\n\n")
            case "3":
                posicao = random.randint(1, 22)
                print(f"\n\nO carro rastreado está na posição {posicao} na corrida.\n\n")
            case "4":
                piloto_equipe = pilotos.sample()
                piloto = piloto_equipe.iloc[0, 1]
                equipe = piloto_equipe.iloc[0, 0]
                print(f"\n\nO nome da equipe é {equipe} e o piloto é o {piloto}.\n\n")
            case "5":
                print("\nSair do programa. Até a próxima!\n")
                break
            case _:
                print("\nOpção Inválida. Tente novamente!\n")

# Tela de BOAS-VINDAS
def boas_vindas():
    print("\n ---- BEM-VINDO AO VISION FE ----")
    print("\n ---- Faça seu cadastro ----\n")

    acao = input("\nDigite 'c' para fazer seu cadastro: ")

    if acao == 'c' or acao == 'C':
        cadastrar_usuarios(usuarios)
        if login(usuarios):
            menu_exclusivo(pilotos)
    else:
        print("Opção inválida. Fim do programa!")

boas_vindas()



            


 ---- BEM-VINDO AO VISION FE ----

 ---- Faça seu cadastro ----

---- CADASTRO ----
Cadastro realizado com sucesso!

-----------------------------------------

Cadastro 1:
    Nome : Diego
    E-mail : diegoc85@me.com
    Senha : 1234


----- LOGIN ----
Login efetuado com sucesso, Diego!

---- Bem-vindo à Área Exclusiva do Vision FE! ----

1) Acesse o Vision FE
2) Mostrar velocidade atual do carro rastreado
3) Posição atual do carro rastreado
4) Nome do piloto e da equipe do carro rastreado
5) Sair


O nome da equipe é NIO 333 Racing e o piloto é o Dan Ticktum.



1) Acesse o Vision FE
2) Mostrar velocidade atual do carro rastreado
3) Posição atual do carro rastreado
4) Nome do piloto e da equipe do carro rastreado
5) Sair


O carro rastreado está na posição 1 na corrida.



1) Acesse o Vision FE
2) Mostrar velocidade atual do carro rastreado
3) Posição atual do carro rastreado
4) Nome do piloto e da equipe do carro rastreado
5) Sair


A velocidade atual do carro rastreado é de 208 km